In [ ]:
from src.misc.helper_functions import api_get_call
import pandas as pd
import time
from datetime import datetime

base_url = 'https://l2beat.com/api/'


origin_keys = ['arbitrum', 'optimism', 'base', 'zksync-era', 'dydx', 'starknet', 'mantle', 'immutablex', 'loopring', 'linea', 'zksync-lite', 'metis', 'polygonzkevm', 'apex', 'nova', 'zkspace', 'sorare', 'rhinofi', 'mantapacific',
               'bobanetwork', 'aevo', 'zora', 'aztecconnect', 'degate2', 'aztec', 'scroll', 'brine', 'publicgoodsnetwork', 'kroma', 'myria', 'canvasconnect', 'cartesi-honeypot', 'fuelv1']

dfMain = pd.DataFrame()
for origin_key in origin_keys:
    url = f"{base_url}/tvl/{origin_key}.json"           

    response_json = api_get_call(url, sleeper=10, retries=20)
    df = pd.json_normalize(response_json['daily'], record_path=['data'], sep='_')

    ## only keep the columns 0 (date) and 1 (total tvl)
    df = df.iloc[:,[0,1]]

    df['date'] = pd.to_datetime(df[0],unit='s')
    df['date'] = df['date'].dt.date
    df.drop(df[df[1] == 0].index, inplace=True)
    df.drop([0], axis=1, inplace=True)
    df.rename(columns={1:'value'}, inplace=True)
    df['metric_key'] = 'tvl'
    df['origin_key'] = origin_key
    max_date = df['date'].max()
    df.drop(df[df.date == max_date].index, inplace=True)
    today = datetime.today().strftime('%Y-%m-%d')
    df.drop(df[df.date == today].index, inplace=True, errors='ignore')
    df.value.fillna(0, inplace=True)
    dfMain = pd.concat([dfMain,df])

    print(f"...loaded for {origin_key}. Shape: {df.shape}")
    time.sleep(1)


In [ ]:
dfMain.to_csv('tvl.csv', index=False)

In [ ]:
## only keep first value of each month
dfMain['month'] = pd.DatetimeIndex(dfMain['date']).month
dfMain['year'] = pd.DatetimeIndex(dfMain['date']).year
dfMain['day'] = pd.DatetimeIndex(dfMain['date']).day
dfMain['first_of_month'] = dfMain['day'] == 1
dfMain = dfMain[dfMain['first_of_month'] == True]
dfMain.drop(['month', 'year', 'day', 'first_of_month'], axis=1, inplace=True)
dfMain.to_csv('tvl_first_of_month.csv', index=False)

In [ ]:
# unpivot origin_key column
test = dfMain.pivot(index='origin_key', columns='date', values='value')

In [ ]:
## test df to csv
test.to_csv('test.csv', index=True)

In [ ]:
prev_val = -700
cur_val = -200

change_val = (prev_val - cur_val) / prev_val
change_val = round(change_val, 4)

print(change_val)

### api.growthepie.xyz

In [ ]:

import requests
import pandas as pd

url = 'https://api.growthepie.xyz/v1/fundamentals.json'
response = requests.get(url)
df = pd.DataFrame(response.json())

df.head(8)

## filter df by metric_key == 'txcount' and origin_key == 'base'
df = df[(df['metric_key'] == 'txcount') & (df['origin_key'] == 'base')]


In [ ]:
## plot metric txcount over date for all origin_keys and order by date
df[(df['metric_key'] == 'txcount') & (df['origin_key'] == 'arbitrum')].sort_values('date').plot(x='date', y='value', figsize=(15, 5), title='Arbitrum Daily Transactions')



## Airtable labelling help

In [ ]:
### AIRTABLE
import pandas as pd
import airtable
import os

#initialize Airtable instance
AIRTABLE_API_KEY = os.getenv("AIRTABLE_API_KEY")
AIRTABLE_BASE_ID = os.getenv("AIRTABLE_BASE_ID")
at = airtable.Airtable(AIRTABLE_BASE_ID, AIRTABLE_API_KEY)

data = pd.DataFrame([c['fields'] for c in at.get('Unlabeled Contracts')['records']])

In [ ]:
from datetime import datetime

from src.db_connector import DbConnector
import src.misc.airtable_functions as at
from eth_utils import to_checksum_address

def read_airtable():
    # read current airtable
    df = at.read_all_airtable()
    if df is None:
        print("Nothing to upload")
    else:
        df['added_on_time'] = datetime.now()
        df.set_index(['address', 'origin_key'], inplace=True)
        # initialize db connection
        db_connector = DbConnector()
        db_connector.upsert_table('blockspace_labels' ,df)


def write_airtable():
    # delete every row in airtable
    at.clear_all_airtable()
    # initialize db connection
    db_connector = DbConnector()
    # get top unlabelled contracts
    df = db_connector.get_unlabelled_contracts('15', '30')
    df['address'] = df['address'].apply(lambda x: to_checksum_address('0x' + bytes(x).hex()))
    # write to airtable
    at.push_to_airtable(df)

read_airtable()
write_airtable()

## web3 test

In [ ]:
from web3 import Web3
import os

tx = '0xdba1ec7832bc857a4f3624557572f24169dda03501fd2b6d6a529647c9a64c51'
url = "https://rpc.zora.energy/"
#url = f"https://rpc.ankr.com/base/{os.getenv('ANKR_API')}"
w3 = Web3(Web3.HTTPProvider(url))

In [ ]:
block = w3.eth.get_block(3913960, full_transactions=True)

In [ ]:
receipt = w3.eth.get_transaction_receipt(tx)

In [ ]:
receipt

In [ ]:
from src.misc.helper_functions import api_post_call
import json
 
payload = {
        "jsonrpc": "2.0",
        "method": "eth_getTransactionReceipt",
        "params": [tx],
        "id": 1
    }
headers = {
    "accept": "application/json",
    "content-type": "application/json"
}

response = api_post_call(url, payload=json.dumps(payload), header=headers)

In [ ]:
block_number_hex = hex(3913960)
payload = {
    "jsonrpc": "2.0",
    "method": "eth_getBlockByNumber",
    "params": [str(block_number_hex), True],
    "id": 1
}
headers = {
    "accept": "application/json",
    "content-type": "application/json"
}

response = api_post_call(url, payload=json.dumps(payload), header=headers)

In [ ]:
from datetime import datetime,timedelta

from src.db_connector import DbConnector
from src.adapters.adapter_raw_rpc import AdapterRPCRaw

adapter_params = {
    'rpc': 'ankr',
    'api_key' : os.getenv("ANKR_API"),
    'chain' : 'base'
}
load_params = {
    'block_start' : 'auto', ## 'auto' or a block number as int
    #'block_start' : 9137631, ## 'auto' or a block number as int
    'batch_size' : 25,
    'threads' : 1
}

# initialize adapter
db_connector = DbConnector()
# initialize adapter
ad = AdapterRPCRaw(adapter_params, db_connector)

## JSON creation

In [ ]:
import os
from src.api.json_creation import JSONCreation
from src.db_connector import DbConnector

db_connector = DbConnector()

json_creator = JSONCreation(os.getenv("S3_CF_BUCKET"), os.getenv("CF_DISTRIBUTION_ID"), db_connector, "v1")
## for testing
#json_creator = JSONCreation(os.getenv("S3_CF_BUCKET"), os.getenv("CF_DISTRIBUTION_ID"), db_connector, "test")

#json_creator.create_all_jsons()

df = json_creator.get_all_data()
#json_creator.create_landingpage_json(df)
#json_creator.create_chain_details_jsons(df)
#json_creator.create_metric_details_jsons(df)
#json_creator.create_fundamentals_json(df)
#json_creator.create_master_json()
#json_creator.create_contracts_json()

In [ ]:
import os
from src.db_connector import DbConnector
from src.api.blockspace_json_creation import BlockspaceJSONCreation
db_connector = DbConnector()
api_version = "v1"

blockspace_json_creator = BlockspaceJSONCreation(os.getenv("S3_CF_BUCKET"), os.getenv("CF_DISTRIBUTION_ID"), db_connector, api_version)
#blockspace_json_creator.create_blockspace_overview_json()
blockspace_json_creator.create_blockspace_comparison_json()

## L2Beat

In [ ]:
from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_l2beat import AdapterL2Beat

adapter_params = {
}
load_params = {
    'origin_keys' : None,
    #'origin_keys' : ['arbitrum'] # see all options in adapter_mapping.py 
}

# initialize adapter
ad = AdapterL2Beat(adapter_params, db_connector)
# extract
df= ad.extract(load_params)
# load
#ad.load(df)

In [ ]:
ad.load(df)

## DefiLlama

In [ ]:
from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_defillama import AdapterDefiLlama

adapter_params = {
}
load_params = {
    'origin_keys' : None,
    #'origin_keys' : ['ethereum'] # see all options in adapter_mapping.py
}

# initialize adapter
ad = AdapterDefiLlama(adapter_params, db_connector)
# extract
df = ad.extract(load_params)
# load
ad.load(df)

## Coingecko

run for projects / chains

In [ ]:
from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_coingecko import AdapterCoingecko

adapter_params = {
}
load_params = {
    'load_type' : 'project',
    'metric_keys' : ['price', 'volume', 'market_cap'],
    'origin_keys' : None,
    #'origin_keys' : ['aptos'], # see all options in adapter_mapping.py
    'days' : 'auto', # auto, max, or a number (as string)
    'vs_currencies' : ['usd', 'eth']
}

# initialize adapter
ad = AdapterCoingecko(adapter_params, db_connector)
# extract
df = ad.extract(load_params)
# load
ad.load(df)

and for imx tokens

In [ ]:
from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_coingecko import AdapterCoingecko

adapter_params = {
}
load_params = {
    'load_type' : 'imx_tokens'
}

# initialize adapter
ad = AdapterCoingecko(adapter_params, db_connector)
# extract
df = ad.extract(load_params)
# load
ad.load(df)

## Dune

In [ ]:
import os
from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_dune import AdapterDune

adapter_params = {
    'api_key' : os.getenv("DUNE_API")
}

load_params = {
    'query_names' : ['rent_paid'], ## fundamentals, waa, stables_mcap, rent_paid
    'days' : 30,
    #'query_names' : None,
    #'days' : 'auto',
}

# initialize adapter
ad = AdapterDune(adapter_params, db_connector)
# extract
df = ad.extract(load_params)
# upload
#ad.load(df)

In [ ]:
ad.load(df)

## Flipside
sometimes some Flipside queries just get stuck -- gotta retrigger them

In [ ]:
import os
from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_flipside import AdapterFlipside

adapter_params = {
    'api_key' : os.getenv("FLIPSIDE_API")
}
load_params = {
    'origin_keys' : ['zksync_era'],
    'metric_keys' : ['stables_mcap'],
    'days' : 'auto',
    # 'origin_keys' : None,
    # 'metric_keys' : None,
    # 'days' : 'auto',
}

# initialize adapter
ad = AdapterFlipside(adapter_params, db_connector)
# extract
df = ad.extract(load_params)
# load
ad.load(df)

In [ ]:
## testing the new flipside api

from flipside import Flipside
flipside = Flipside("74c43ebc-3291-4953-8aeb-65640da7c852", "https://api-v2.flipsidecrypto.xyz")

sql = """
 select 
    BLOCK_NUMBER, BLOCK_TIMESTAMP, BLOCK_HASH, TX_HASH, NONCE, POSITION, ORIGIN_FUNCTION_SIGNATURE, FROM_ADDRESS, TO_ADDRESS, ETH_VALUE, TX_FEE, GAS_PRICE, GAS_LIMIT, 
    GAS_USED, L1_GAS_PRICE, L1_GAS_USED, L1_FEE_SCALAR, L1_SUBMISSION_BATCH_INDEX, L1_SUBMISSION_TX_HASH, L1_STATE_ROOT_BATCH_INDEX, 
    L1_STATE_ROOT_TX_HASH, CUMULATIVE_GAS_USED, INPUT_DATA, STATUS
from optimism.core.fact_transactions
where block_number >= 104130000
and block_number < 104135000
order by block_number asc
"""

# Run the query against Flipside's query engine and await the results
query_result_set = flipside.query(sql)

## Zettablock

In [ ]:
import os
from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_zettablock import AdapterZettablock

adapter_params = {
    'api_key' : os.getenv("ZETTABLOCK_API")
}
load_params = {
    #'origin_keys' : ['zksync_era', 'polygon_zkevm'],
    #'metric_keys' : ['txcount', ''],
    'days' : 10,
    'origin_keys' : None,
    'metric_keys' : None,
    # 'days' : 'auto',
}

# initialize adapter
ad = AdapterZettablock(adapter_params, db_connector)
# extract
df = ad.extract(load_params)
# load
ad.load(df)

## RPC raw data load

In [ ]:
import os
from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_raw_rpc import AdapterRPCRaw

adapter_params = {
    'rpc': 'ankr',
    'api_key' : os.getenv("ANKR_API"),
    'chain' : 'base'
}

load_params = {
    'block_start' : 'auto', ## 'auto' or a block number as int
    #'block_start' : 9137631, ## 'auto' or a block number as int
    'batch_size' : 10,
    'threads' : 5
}

# initialize adapter
ad = AdapterRPCRaw(adapter_params, db_connector)
# extract
ad.extract_raw(load_params)

In [ ]:
import os
from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_raw_rpc import AdapterRPCRaw

adapter_params = {
    'rpc': 'ankr',
    'api_key' : os.getenv("ANKR_API"),
    'chain' : 'optimism'
}

load_params = {
    'block_start' : 'auto', ## 'auto' or a block number as int
    #'block_start' : 9137631, ## 'auto' or a block number as int
    'batch_size' : 10,
    'threads' : 5
}

# initialize adapter
ad = AdapterRPCRaw(adapter_params, db_connector)
# extract
ad.extract_raw(load_params)

## Adapter Nader

In [ ]:
import os
from src.adapters.adapter_nader import BaseNodeAdapter
from src.db_connector import DbConnector
adapter_params = {
    'rpc': 'local_node',
    'chain': 'base',
    'node_url': os.getenv("BASE_NODE"),
}

# Initialize DbConnector
db_connector = DbConnector()

# Initialize BaseNodeAdapter
adapter = BaseNodeAdapter(adapter_params, db_connector)

# Test database connectivity
if not adapter.check_db_connection():
    print("Failed to connect to database.")
else:
    print("Successfully connected to database.")

# Test S3 connectivity
if not adapter.check_s3_connection():
    print("Failed to connect to S3.")
else:
    print("Successfully connected to S3.")

# Test run method
load_params = {
    'block_start': 'auto',
    'batch_size': 10,
    'threads': 5,
}

adapter.extract_raw(load_params)

## ZettaBlock raw data load

In [ ]:
import os
from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_raw_zettablock import AdapterZettaBlockRaw

adapter_params = {
    'api_key' : os.getenv("ZETTABLOCK_API")
}

load_params = {
    #'keys' : ['polygon_zkevm_tx', 'zksync_era_tx'],
    'keys' : ['zksync_era_tx'],
    #'block_start' : 'auto', ## 'auto' or a block number as int
    'block_start' : 9137631, ## 'auto' or a block number as int
}

# initialize adapter
ad = AdapterZettaBlockRaw(adapter_params, db_connector)
# extract
ad.extract_raw(load_params, if_exists = 'ignore')

## Chainbase raw

In [ ]:
import os
from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_raw_chainbase import AdapterChainbaseRaw

adapter_params = {
    'api_key' : os.getenv("CHAINBASE_API")
}

load_params = {
    'keys' : ['arbitrum_tx'],
    #'block_start' : 'auto', ## 'auto' or a block number as int
    'block_start' : 64900000, ## until 65,570,000
}

# initialize adapter
ad = AdapterChainbaseRaw(adapter_params, db_connector)
# extract
ad.extract_raw(load_params)

## IMX raw data load

In [ ]:
## implement orchestration?

from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_raw_imx import AdapterRawImx

adapter_params = {
    'load_types' : ['withdrawals', 'deposits', 'trades', 'orders_filled', 'transfers', 'mints'],
    'forced_refresh' : 'no',

    #'load_types' : ['orders_filled'],
    #'forced_refresh' : '2023-04-01T00:00:00.000000Z',
}

# initialize adapter
ad = AdapterRawImx(adapter_params, db_connector)
# extract raw (and load raw in case of IMX)
df_raw = ad.extract_raw()

## Flipside raw data load

In [ ]:
import os
from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_raw_flipside import AdapterFlipsideRaw

adapter_params = {
    'api_key' : os.getenv("FLIPSIDE_API")
}

load_params = {
    'keys' : ['arbitrum_tx', 'optimism_tx'],
    'block_start' : 'auto',
}

# initialize adapter
ad = AdapterFlipsideRaw(adapter_params, db_connector)
# extract
df = ad.extract_raw(load_params)

## SQL aggregation

In [1]:
## Loads currently IMX txcount, daa, fees_paid
## also loads user_base_weekly

from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_sql import AdapterSQL

adapter_params = {
}

load_params = {
    'load_type' : 'metrics', ## usd_to_eth or metrics or blockspace
    
    #'days' : 'auto', ## days as int our 'auto
    'origin_keys' : None, ## origin_keys as list or None
    #'metric_keys' : None, ## metric_keys as list or None

    'days' : 200, ## days as int our 'auto
    #'origin_keys' : ['zora', 'gitcoin_pgn'], ## origin_keys as list or None
    'metric_keys' : ['user_base_weekly'], ## metric_keys as list or None
}

# initialize adapter
ad = AdapterSQL(adapter_params, db_connector)

# extract
df = ad.extract(load_params)
# # load
ad.load(df)

Connecting to orbal_writer@orbal-main.cydw1x28knil.us-east-1.rds.amazonaws.com:5432
Adapter SQL Aggregation initialized with {}.
... executing query: user_base_weekly - multi with {'Days': 200, 'aggregation': 'week'} days
...query loaded: user_base_weekly multi with 200 days. DF shape: (252, 4)
SQL Aggregation extract done for {'load_type': 'metrics', 'origin_keys': None, 'days': 200, 'metric_keys': ['user_base_weekly']}. DataFrame shape: (252, 1)
Load SQL Aggregation done - 252 rows upserted in fact_kpis


In [ ]:
from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_sql import AdapterSQL

adapter_params = {
}

load_params = {
    'load_type' : 'profit', ## usd_to_eth or metrics or blockspace or profit
    'days' : 30, ## days as int
    'origin_keys' : None, ## origin_keys as list or None
    'metric_keys' : None, ## metric_keys as list or None
}

# initialize adapter
ad = AdapterSQL(adapter_params, db_connector)

# extract
df = ad.extract(load_params)
# # load
ad.load(df)

In [ ]:
from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_sql import AdapterSQL

adapter_params = {
}

load_params = {
    'load_type' : 'usd_to_eth', ## usd_to_eth or metrics or blockspace
    'days' : 30, ## days as int
    'origin_keys' : None, ## origin_keys as list or None
    'metric_keys' : None, ## metric_keys as list or None
}

# initialize adapter
ad = AdapterSQL(adapter_params, db_connector)

# extract
df = ad.extract(load_params)
# # load
ad.load(df)

In [1]:
from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_sql import AdapterSQL

adapter_params = {
}

load_params = {
    'load_type' : 'blockspace', ## usd_to_eth or metrics or blockspace
    'days' : '300', ## days as or auto
    #'origin_keys' : ['arbitrum', 'zksync_era', 'polygon_zkevm', 'optimism', 'base', 'imx'], ## origin_keys as list or None
    'origin_keys' : ['linea']
}

# initialize adapter
ad = AdapterSQL(adapter_params, db_connector)

# extract
ad.extract(load_params)

Connecting to orbal_writer@orbal-main.cydw1x28knil.us-east-1.rds.amazonaws.com:5432
Adapter SQL Aggregation initialized with {}.
...aggregating contract data for linea and last 300 days...
...upserting contract data for linea. Total rows: 71125...
...aggregating total usage for linea and last 300 days...
...upserting total usage usage for linea. Total rows: 128...
...aggregating native_transfers for linea and last 300 days...
...upserting native_transfers for linea. Total rows: 127...
...aggregating smart_contract_deployments for linea and last 300 days...
...upserting smart_contract_deployments for linea. Total rows: 127...
...aggregating sub categories for linea and last 5000 days...
...upserting sub categories for linea. Total rows: 0...
...aggregating unlabeled usage for linea and last 5000 days...
...upserting unlabeled usage for linea. Total rows: 128...
Finished loading blockspace queries for linea
Finished loading blockspace for all chains


### Blockspace logic
- for each chain, aggregate the daily contracts usage


In [ ]:
from src.db_connector import DbConnector
from src.misc.helper_functions import get_missing_days_blockspace
db_connector = DbConnector()

chain_list = ['optimism']
#chain_list = ['arbitrum', 'zksync_era', 'polygon_zkevm', 'optimism']

In [ ]:
for chain in chain_list:
    #days = get_missing_days_blockspace(db_connector, chain)
    days = 10000

    # ## aggregate contract data
    # print(f"aggregating contract data for {chain} and last {days} days...")
    # df = db_connector.get_blockspace_contracts(chain, days)
    # df.set_index(['address', 'date', 'origin_key'], inplace=True)

    # print(f"upserting contract data for {chain}. Total rows: {df.shape[0]}...")
    # db_connector.upsert_table('blockspace_fact_contract_level', df)

    # ## determine total usage
    # print(f"aggregating total usage for {chain} and last {days} days...")
    # df = db_connector.get_blockspace_total(chain, days)
    # df.set_index(['date', 'sub_category_key' ,'origin_key'], inplace=True)

    # print(f"upserting total usage usage for {chain}. Total rows: {df.shape[0]}...")
    # db_connector.upsert_table('blockspace_fact_sub_category_level', df)

    # ## aggregate native transfers
    # print(f"aggregating native_transfers for {chain} and last {days} days...")
    # df = db_connector.get_blockspace_native_transfers(chain, days)
    # df.set_index(['date', 'sub_category_key' ,'origin_key'], inplace=True)

    # print(f"upserting native_transfers for {chain}. Total rows: {df.shape[0]}...")
    # db_connector.upsert_table('blockspace_fact_sub_category_level', df)

    # ## aggregate contract deployments
    # print(f"aggregating smart_contract_deployments for {chain} and last {days} days...")
    # df = db_connector.get_blockspace_contract_deplyments(chain, days)
    # df.set_index(['date', 'sub_category_key' ,'origin_key'], inplace=True)

    # print(f"upserting smart_contract_deployments for {chain}. Total rows: {df.shape[0]}...")
    # db_connector.upsert_table('blockspace_fact_sub_category_level', df)

    # ALL below needs to be retriggerd when mapping changes (e.g. new addresses got labeled or new categories added etc.)
    ## aggregate by sub categories
    print(f"aggregating sub categories for {chain} and last {days} days...")
    df = db_connector.get_blockspace_sub_categories(chain, days)
    df.set_index(['date', 'sub_category_key' ,'origin_key'], inplace=True)

    print(f"upserting sub categories for {chain}. Total rows: {df.shape[0]}...")
    db_connector.upsert_table('blockspace_fact_sub_category_level', df)

    ## determine unlabeled usage
    print(f"aggregating unlabeled usage for {chain} and last {days} days...")
    df = db_connector.get_blockspace_unlabeled(chain, days)
    df.set_index(['date', 'sub_category_key' ,'origin_key'], inplace=True)

    print(f"upserting unlabeled usage for {chain}. Total rows: {df.shape[0]}...")
    db_connector.upsert_table('blockspace_fact_sub_category_level', df)


# days = get_missing_days_blockspace(db_connector, 'imx')

# df = db_connector.get_blockspace_imx(days)
# df.set_index(['date', 'sub_category_key' ,'origin_key'], inplace=True)

# print(f"...upserting imx data . Total rows: {df.shape[0]}...")
# db_connector.upsert_table('blockspace_fact_sub_category_level', df)

In [ ]:
from src.db_connector import DbConnector
db_connector = DbConnector()

In [ ]:
df = db_connector.get_top_contracts_by_category('main_category', 'unlabeled', 'arbitrum', 'gas', 7)

In [ ]:
print(df.to_markdown())

## S3

In [ ]:
## connect to s3 bucket and output list of files
import pandas as pd
import boto3
import os

s3 = boto3.resource(
    service_name='s3',
    region_name='us-east-1',
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY")
)

bucket = s3.Bucket('gtp-longterm')

## get list of files in bucket with last modified date
files = []
for obj in bucket.objects.all():
    files.append([obj.key, obj.last_modified])

df = pd.DataFrame(files, columns=['key', 'last_modified'])

## filter out files where key starts with 'imx'
df = df[~df.key.str.startswith('imx')]

df['chain'] = df.key.str.split('/').str[0]

## create new column block_range that extracts the string between 'tx_' and '.parquet' in the key column using lambda function
df['block_range'] = df.key.apply(lambda x: x[x.find('tx_')+3:x.find('.parquet')])
df['block_start'] = df.block_range.str.split('-').str[0].astype(int)
df['block_end'] = df.block_range.str.split('-').str[1].astype(int)

## sort by block_start
df.sort_values(by='block_start', inplace=True, ascending=True)

In [ ]:
df

### Arbitrum

In [ ]:
df_arbitrum = df[df.chain == 'arbitrum']

In [ ]:
df_arbitrum

In [ ]:
## load first file in df_arbitrum into df
df_arbitrum_flipside = pd.read_parquet(f"s3://gtp-longterm/{df_arbitrum.key.iloc[0]}")
df_arbitrum_flipside.sort_values(by='ETH_VALUE', inplace=True, ascending=False)
df_arbitrum_flipside.head(10)

In [ ]:
## load first file in df_arbitrum with block_start >= 96528665 into df
df_arbitrum_chainbase = pd.read_parquet(f"s3://gtp-longterm/{df_arbitrum[df_arbitrum.block_start >= 96528665].key.iloc[10]}")
df_arbitrum_chainbase.head()

### Optimism

In [ ]:
df_optimism = df[df.chain == 'optimism']

In [ ]:
## load first file in df_arbitrum into df
df_optimism_flipside = pd.read_parquet(f"s3://gtp-longterm/{df_optimism.key.iloc[0]}")
df_optimism_flipside.head()

In [ ]:
# df_optimism_flipside[['TX_HASH', 'ETH_VALUE', 'TX_FEE', 'GAS_PRICE', 'GAS_LIMIT', 'GAS_USED',
#        'L1_GAS_PRICE', 'L1_GAS_USED', 'L1_FEE_SCALAR',  'CUMULATIVE_GAS_USED' ]]

## filter df_optimism_flipside where TX_HASH = 0xA5E0DACD8D29020C04139F8506C01CBA8B6E561CE567DF8DA35857722232F559 and select columns 'TX_HASH', 'ETH_VALUE', 'TX_FEE', 'GAS_PRICE', 'GAS_LIMIT', 'GAS_USED','L1_GAS_PRICE', 'L1_GAS_USED', 'L1_FEE_SCALAR',  'CUMULATIVE_GAS_USED' 

df_optimism_flipside[df_optimism_flipside.TX_HASH == '0xA5E0DACD8D29020C04139F8506C01CBA8B6E561CE567DF8DA35857722232F559'.lower()][['TX_HASH', 'ETH_VALUE', 'TX_FEE', 'GAS_PRICE', 'GAS_LIMIT', 'GAS_USED','L1_GAS_PRICE', 'L1_GAS_USED', 'L1_FEE_SCALAR',  'CUMULATIVE_GAS_USED' ]]

In [ ]:
df_optimism_flipside['GAS_PRICE'] / 1e9

In [ ]:
df_optimism_flipside.columns

In [ ]:
## load first file in df_arbitrum with block_start >= 96528665 into df
df_optimism_chainbase = pd.read_parquet(f"s3://gtp-longterm/{df_optimism[df_optimism.block_start >= 103428989].key.iloc[0]}")
df_optimism_chainbase.head()

### Polygon zkEVM

In [ ]:
df_polygon = df[df.chain == 'polygon_zkevm']

In [ ]:
## load first file in df_arbitrum into df
df_polygon_zb = pd.read_parquet(f"s3://gtp-longterm/{df_polygon.key.iloc[0]}")
df_polygon_zb.head()

### zkSync Era

In [ ]:
df_zksync = df[df.chain == 'zksync_era']

In [ ]:
## load first file in df_arbitrum into df
df_zksync_zb = pd.read_parquet(f"s3://gtp-longterm/{df_zksync.key.iloc[0]}")
df_zksync_zb.head()

In [ ]:
# ## rename files in S3 bucket that contain 'block_'
# for index, row in df[df.key.str.contains('block_')].iterrows():
#     print(row['key'])
#     old_key = row['key']
#     new_key = old_key.replace('block_', '')
#     print(new_key)
#     s3.Object('gtp-longterm', new_key).copy_from(CopySource='gtp-longterm/'+old_key)
#     #s3.Object('gtp-longterm', old_key).delete()


In [ ]:
df_arbitrum[df_arbitrum.block_start >= 96528665].sort_values(by='last_modified', ascending=True)

In [ ]:
main_zksync()

## indexed.xyz

In [ ]:
import boto3

aws_access_key_id = "43c31ff797ec2387177cabab6d18f15a"
aws_secret_access_key = "afb354f05026f2512557922974e9dd2fdb21e5c2f5cbf929b35f0645fb284cf7"
bucket_name = 'indexed-xyz'

s3 = boto3.client(
        's3',
        aws_access_key_id=aws_access_key_id,
        aws_secret_access_key=aws_secret_access_key
        )


In [ ]:
import boto3

# Set the profile name
profile_name = 'indexedxyz'

# Create an S3 client using the profile
session = boto3.Session(profile_name=profile_name)
s3 = session.client('s3')

In [ ]:
#test s3 connection
response = s3.list_objects(Bucket=bucket_name)

In [ ]:
## list all files in bucket
for obj in s3.list_objects(Bucket=bucket_name)['Contents']:
    print(obj['Key'])

In [ ]:
from src.adapters.adapter_raw_gtp import NodeAdapter
from src.db_connector import DbConnector
from datetime import datetime,timedelta
from airflow.decorators import dag, task 
import os


adapter_params = {
    'rpc': 'local_node',
    'chain': 'zora',
    'node_url': os.getenv("ZORA_RPC"),
}

# Initialize DbConnector
db_connector = DbConnector()

# Initialize NodeAdapter
adapter = NodeAdapter(adapter_params, db_connector)

# Test database connectivity
if not adapter.check_db_connection():
    print("Failed to connect to database.")
else:
    print("Successfully connected to database.")

# Test S3 connectivity
if not adapter.check_s3_connection():
    print("Failed to connect to S3.")
else:
    print("Successfully connected to S3.")
    
# Extract
load_params = {
    'block_start': '4982120',
    'batch_size': 1,
    'threads': 1,
}
adapter.extract_raw(load_params)



In [6]:
from src.db_connector import DbConnector
db_connector = DbConnector()

def hex_to_bytea(hex_string):
    # Remove '0x' prefix if present
    if hex_string.startswith('0x'):
        hex_string = hex_string[2:]
    # Convert to bytes
    return bytes.fromhex(hex_string)

Connecting to lorenz@orbal-main.cydw1x28knil.us-east-1.rds.amazonaws.com


In [7]:
import pandas as pd
import datetime
csv_file_path = 'linea.csv'
#address,origin_key,contract_name,project_name,sub_category_key,labelling_type
df = pd.read_csv(csv_file_path)
df['added_on_time'] = datetime.datetime.now()
df['address'] = df['address'].apply(hex_to_bytea)
df.set_index(['address', 'origin_key'], inplace=True)

In [9]:
# Specify the table name
table_name = 'blockspace_labels'

# Upload the CSV file to the table
db_connector.upsert_table(table_name, df)

45

In [8]:
df

,,contract_name,project_name,sub_category_key,labelling_type,added_on_time
address,origin_key,,,,,
b'\xa6Xt-3\xeb\xd2\xce/\x0b\xdf\xf75\x15\xaay\x7f\xd1a\xd9',linea,Relayer,LayerZero,cc_communication,Lorenz,2023-11-22 08:31:04.687907
b'\x80\xe3\x82\x91\xe0c9\xd1\n\xabH<ei]\x00M\xbd\\i',linea,Router,SyncSwap,dex,Lorenz,2023-11-22 08:31:04.687907
b'\x9d\xdan\xf3\xd9\x19\xc9\xbc\x88\x85\xd5V\t\x99\xa3d\x041\xe8\xe6',linea,Swap,MetaMask,dex,Lorenz,2023-11-22 08:31:04.687907
b'\x871\xd5N\x9d\x02\xc2\x86v}V\xac\x03\xe8\x03|\x07\xe0\x1e\x98',linea,RouterETH,Stargate,bridge,Lorenz,2023-11-22 08:31:04.687907
"b""'.\x15m\xf8\xdaQ<i\xcbA\xccz\x99\x18]S\xf9&\xbb""",linea,Router,Horizen,dex,Lorenz,2023-11-22 08:31:04.687907
"b'P\x8c\xa8-\xf5f\xdc\xd1\xb0\xde\x82\x96\xe7\n\x963,\xd6D\xec'",linea,L2MessageService,Linea,cc_communication,Lorenz,2023-11-22 08:31:04.687907
b'\x00\x9a\x0b|8\xb5B \x896\xf1\x17\x91Q\xcd\x08\xe2\x9483',linea,Core,Lineabank,lending,Lorenz,2023-11-22 08:31:04.687907
b'\xd5nN\xab#\xcb\x81\xf41h\xf9\xf4R\x11\xeb\x02{\x9a\xc7\xcc',linea,VerifierNetwork,LayerZero,cc_communication,Lorenz,2023-11-22 08:31:04.687907
b'a1\xb5\xfa\xe1\x9e\xa4\xf9\xd9d\xea\xc0@\x8eD\x08\xb6c7\xb5',linea,MetaAggregationRouterV2,KyberSwap,dex,Lorenz,2023-11-22 08:31:04.687907
